In [ ]:
# import libraries
import numpy as np
from imageio import imread

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib

# path to ffmpeg for creating more efficiently encoded videos
matplotlib.rcParams['animation.ffmpeg_path'] = r'C:\\Users\\dd\\.conda\\envs\\ML_in_geosciences\\ffmpeg\\bin\\ffmpeg.exe'

In [ ]:
# create colormap to use for plotting ndfi
colors = ["darkorange", "lawngreen"]
nodes = [0., 1.]
CMAP = matplotlib.colors.LinearSegmentedColormap.from_list("cmap_ndfi", list(zip(nodes, colors)))

In [ ]:
def subplot_setup(ax, img_background, img_first, title):
    """Sets up individual subplots with the first frame and plot styling."""
    # show background of first frame
    ax.imshow(img_background)
    # show first observation
    img = ax.imshow(img_first, cmap=CMAP)
    
    # style the subplot
    ax.set_axis_off()
    ax.set_title(title)
    
    return img

def animate(i):
    #line.set_ydata(np.sin(x + i / 50))  # update the data.
    #return line,
    return ax1.imshow(DATA[:,:,i], cmap=CMAP), ax2.imshow(DATA2[:,:,i], cmap = CMAP), ax3.imshow(DATA3[:,:,i], cmap = CMAP),

In [ ]:
def load_dataset(inpath):
    """Load raster and convert it into numpy array with values between -1 and 1."""
    loaded_data = imread(rasterpath)
    print(loaded_data.shape)
    loaded_data = loaded_data / 10_000
    loaded_data[loaded_data<-3] = np.nan
    loaded_data[loaded_data<-1] = -1
    loaded_data[loaded_data>1] = 1
    return loaded_data

rasterpath = 'E:/__Earth_Observation/grasslands/data/raster/ndfi/ndfi_branden.tif'
DATA = load_dataset(rasterpath)
DATA2= DATA
DATA3= DATA

In [ ]:
plt.hist(DATA.flatten())

In [ ]:
# create figure and style it
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(20, 5), dpi=300.)
fig.suptitle('NDFI throughout 2018 on three sites in Germany:')

# set up individual subplots
# first subplot returns a Mappable which is used for creating the colorbar
img = subplot_setup(ax1, DATA[:,:,0], DATA[:,:,0], 'Hello world')
subplot_setup(ax2, DATA2[:,:,0], DATA2[:,:,0], 'Hello world2')
subplot_setup(ax3, DATA3[:,:,0], DATA3[:,:,0], 'Hello world3')

# create colorbar
cbar = plt.colorbar(img, ax=[ax1, ax2, ax3])
cbar.set_label('NDFI')


# animate the figure
ani = animation.FuncAnimation(
    fig, animate, frames=DATA.shape[2], interval=20, blit=True, save_count=50)

In [ ]:
# To save the animation, use e.g.
writer = animation.FFMpegWriter(
    fps=2, metadata=dict(artist='Me'))#, bitrate=1800)
ani.save("C:\\users\\dd\\downloads\\movie_ndfi_test_3.mp4", writer=writer)

In [ ]:
f'<html>\n<body>\n{0+2}\n</body>\n</html>'

In [ ]:
html_str = f'{ani.to_html5_video()}'